In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag = 'mlp_with_split_1pc_usampled_bnorm_on_encoder_only'
loss = 'emd'
n_pc_points = 2048
random_seed = 42
bneck_list = [128]
class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]
in_data  = load_multiple_version_of_pcs('uniform_one' ,syn_id, n_classes=1)

In [15]:
train_params = default_train_params()

for bneck in bneck_list:
    experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
    train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
    create_dir(train_dir)

    reset_tf_graph()    
    encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck)    
    conf = Conf(n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 500,
                batch_size = train_params['batch_size'],
                denoising = False,
                learning_rate = train_params['learning_rate'],
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = train_params['saver_step'],
                z_rotate = False,
                encoder = encoder,
                decoder = decoder,
                encoder_args = enc_args,
                decoder_args = dec_args
               )
    print conf
    conf.experiment_name = 'experiment_' + str(experiment_id)
    conf.held_out_step = 5
    conf.save(osp.join(train_dir, 'configuration'))
    ae = PointNetAutoEncoder(conf.experiment_name, conf)

    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['test'])
    fout.close()

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/02691156/85_5_10/test.txt
405 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/02691156/85_5_10/train.txt
3438 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/02691156/85_5_10/val.txt
202 pclouds were loaded. They belong in 1 shape-classes.
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm_finish': False, 'verbose': True, 'b_norm': False, 'layer_sizes': [256, 256, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1], 'n_filters': [64, 128, 128, 256, 12